In [1]:
import pandas as pd

df = pd.read_csv("../data/raw/sentimentdataset.csv")
df.head()


,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [4]:
df.columns


Index(['Unnamed: 0.1', 'Unnamed: 0', 'Text', 'Sentiment', 'Timestamp', 'User',
       'Platform', 'Hashtags', 'Retweets', 'Likes', 'Country', 'Year', 'Month',
       'Day', 'Hour'],
      dtype='object')

In [5]:
df = df.drop(columns=["Unnamed: 0", "Unnamed: 0.1"], errors="ignore")
df.columns


Index(['Text', 'Sentiment', 'Timestamp', 'User', 'Platform', 'Hashtags',
       'Retweets', 'Likes', 'Country', 'Year', 'Month', 'Day', 'Hour'],
      dtype='object')

In [6]:
import sys
sys.path.append("../")

from src.preprocessing import clean_text

df["clean_text"] = df["Text"].apply(clean_text)
df[["Text", "clean_text"]].head()


,Text,clean_text
0,Enjoying a beautiful day at the park! ...,enjoying a beautiful day at the park
1,Traffic was terrible this morning. ...,traffic was terrible this morning
2,Just finished an amazing workout! 💪 ...,just finished an amazing workout
3,Excited about the upcoming weekend getaway! ...,excited about the upcoming weekend getaway
4,Trying out a new recipe for dinner tonight. ...,trying out a new recipe for dinner tonight


In [7]:
from src.sentiment import get_sentiment_score, get_sentiment_label

df["sentiment_score"] = df["clean_text"].apply(get_sentiment_score)
df["predicted_sentiment"] = df["sentiment_score"].apply(get_sentiment_label)

df.head()


,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,clean_text,sentiment_score,predicted_sentiment
0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12,enjoying a beautiful day at the park,0.8074,Positive
1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8,traffic was terrible this morning,-0.4767,Negative
2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15,just finished an amazing workout,0.5859,Positive
3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18,excited about the upcoming weekend getaway,0.3400,Positive
4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19,trying out a new recipe for dinner tonight,0.0000,Neutral


In [8]:
df["predicted_sentiment"].value_counts()


predicted_sentiment
Positive    468
Negative    182
Neutral      82
Name: count, dtype: int64

In [9]:
df.groupby("predicted_sentiment")["sentiment_score"].mean()


predicted_sentiment
Negative   -0.484738
Neutral     0.000000
Positive    0.606225
Name: sentiment_score, dtype: float64

In [10]:
pd.crosstab(df["Sentiment"], df["predicted_sentiment"])


predicted_sentiment,Negative,Neutral,Positive
Sentiment,,,
Acceptance,0,0,3
Acceptance,0,0,5
Accomplishment,0,0,3
Admiration,0,0,1
Admiration,0,0,2
...,...,...,...
Wonder,0,1,0
Wonder,0,1,0
Wonderment,0,0,1


In [11]:
df.to_csv("../data/processed/sentiment_results.csv", index=False)
